In [96]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

# 定義一個函數來從檔案名稱中提取日期
def extract_date_from_filename(filename):
    match = re.search(r'(\d+\.\d+\.\d+)', filename)
    if match:
        return match.group(1)
    return None

# 讀取 CSV 文件
prev_file = 'Meta Backlog_9.12.2024.csv'
current_file = 'Meta Backlog_9.15.2024.csv'

df_prev = pd.read_csv(prev_file)
df_curr = pd.read_csv(current_file)

# 從文件名中提取日期
previous_date = extract_date_from_filename(prev_file)
current_date = extract_date_from_filename(current_file)

print(f"已成功讀取 CSV 文件")
print(f"前一個日期: {previous_date}")
print(f"當前日期: {current_date}")




已成功讀取 CSV 文件
前一個日期: 9.12.2024
當前日期: 9.15.2024


In [97]:
# 定義處理 DataFrame 的函數
def process_dataframe(df, date_str):
    # 將日期字符串轉換為 datetime 對象
    today = datetime.strptime(date_str, '%m.%d.%Y')
    
    # 將 CMAD Date 轉換為 datetime 對象
    df['CMAD Date'] = pd.to_datetime(df['CMAD Date'])
    
    # 過濾出大於或等於今天的行
    future_rows = df[df['CMAD Date'] >= today]
    
    # 找出最接近今天的未來日期
    nearest_future_date = future_rows['CMAD Date'].min()
    
    # 收集所有具有這個最接近未來日期的行
    result_rows = future_rows[future_rows['CMAD Date'] == nearest_future_date]
    
    return result_rows


In [98]:
# 處理兩個 DataFrame
previous_CMAD_rows = process_dataframe(df_prev, previous_date)
current_CMAD_rows = process_dataframe(df_curr, current_date)

print("已處理兩個 DataFrame")


已處理兩個 DataFrame


In [99]:
# # 印出 previous_CMAD_rows 和 current_CMAD_rows
# print("previous_CMAD_rows:")
# print(previous_CMAD_rows)
# print("\ncurrent_CMAD_rows:")
# print(current_CMAD_rows)


In [100]:
# 創建 result.csv 檔案，複製 df_prev 的內容
df_result = df_prev.copy()
# 重新排列欄位順序
columns = df_result.columns.tolist()

# 找到 'CMAD Date' 和 'Scheduled Quantity' 的索引
cmad_date_index = columns.index('CMAD Date')
scheduled_quantity_index = columns.index('Scheduled Quantity')
# 找到 'Delivery' 的索引
delivery_index = columns.index('Delivery')

# 按照正確的順序插入新列
if 'Today CMAD Date' not in df_result.columns:
    columns.insert(cmad_date_index + 1, 'Today CMAD Date')
if 'Today Delivery' not in df_result.columns:
    columns.insert(columns.index('Delivery') + 1, 'Today Delivery')
if 'Today Scheduled Quantity' not in df_result.columns:
    columns.insert(columns.index('Scheduled Quantity') + 1, 'Today Scheduled Quantity')
# 重新排列 DataFrame 的欄位
df_result = df_result.reindex(columns=columns)
df_result.to_csv('result.csv', index=False)

print("已創建 result.csv 檔案並添加新欄位")


已創建 result.csv 檔案並添加新欄位


In [101]:
# 遍歷 df_result 的每一行
for index, row in df_result.iterrows():
    # 在 current_CMAD_rows 中查找匹配的 Sales Document
    match = current_CMAD_rows[current_CMAD_rows['Sales Document'] == row['Sales Document']]
    
    if not match.empty:
        # 如果找到匹配項
        current_row = match.iloc[0]
        
        # 檢查 CMAD Date 是否有變化
        if current_row['CMAD Date'] != row['CMAD Date']:
            # 將日期格式轉換為 "M/D/YYYY"
            formatted_date = current_row['CMAD Date'].strftime('%m/%d/%Y')
            df_result.at[index, 'Today CMAD Date'] = formatted_date
            print(f"Sales Document {row['Sales Document']} 的 CMAD Date 從 {row['CMAD Date']} 變更為 {formatted_date}")

            
        
        # 檢查 Scheduled Quantity 是否有變化
        if current_row['Scheduled Quantity'] != row['Scheduled Quantity']:
            df_result.at[index, 'Today Scheduled Quantity'] = current_row['Scheduled Quantity']
            # print the change
            print(f"Sales Document {row['Sales Document']} 的 Scheduled Quantity 從 {row['Scheduled Quantity']} 變更為 {current_row['Scheduled Quantity']}")
        # 檢查 current row 是否有 Delivery
        if 'Delivery' in current_row and pd.notna(current_row['Delivery']):
            df_result.at[index, 'Today Delivery'] = current_row['Delivery']
            # print(f"Sales Document {row['Sales Document']} 的 Today Delivery 更新為 {current_row['Delivery']}")
print("已完成比對和更新")


Sales Document 3000169794 的 Scheduled Quantity 從 1098 變更為 252
Sales Document 3000183284 的 CMAD Date 從 NaT 變更為 09/16/2024
Sales Document 3000183284 的 Scheduled Quantity 從 0 變更為 408
Sales Document 3000183284 的 CMAD Date 從 2024-09-13 00:00:00 變更為 09/16/2024
Sales Document 3000183284 的 Scheduled Quantity 從 130 變更為 408
Sales Document 3000183284 的 CMAD Date 從 2024-09-20 00:00:00 變更為 09/16/2024
Sales Document 3000183284 的 Scheduled Quantity 從 278 變更為 408
Sales Document 3000183284 的 CMAD Date 從 NaT 變更為 09/16/2024
Sales Document 3000183284 的 Scheduled Quantity 從 0 變更為 408
Sales Document 3000183284 的 CMAD Date 從 2024-09-20 00:00:00 變更為 09/16/2024
Sales Document 3000183284 的 Scheduled Quantity 從 1932 變更為 408
Sales Document 3000183284 的 CMAD Date 從 NaT 變更為 09/16/2024
Sales Document 3000183284 的 Scheduled Quantity 從 0 變更為 408
Sales Document 3000183284 的 CMAD Date 從 2024-09-27 00:00:00 變更為 09/16/2024
Sales Document 3000183284 的 Scheduled Quantity 從 6264 變更為 408
Sales Document 3000177489 的 CMAD Date 

/var/folders/w2/7ts44mgn7xxffrh8h14pznt40000gn/T/ipykernel_65816/1241462681.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '09/16/2024' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_result.at[index, 'Today CMAD Date'] = formatted_date


In [102]:
changed_rows = df_result[(df_result['Today CMAD Date'].notna()) | (df_result['Today Scheduled Quantity'].notna())]
print("有變化的行：")
print(changed_rows[['Sales Document', 'CMAD Date', 'Today CMAD Date', 'Scheduled Quantity', 'Today Scheduled Quantity']])

# 顯示整個更新後的 DataFrame
print("\n更新後的 df_result：")
print(df_result)

# 將更新後的結果再次寫入 result.csv 檔案
df_result.to_csv('result.csv', index=False)
print("已將更新後的結果寫入 result.csv 檔案")

有變化的行：
     Sales Document  CMAD Date Today CMAD Date  Scheduled Quantity  \
29       3000169794 2024-09-16             NaN                1098   
50       3000183284        NaT      09/16/2024                   0   
51       3000183284 2024-09-13      09/16/2024                 130   
52       3000183284 2024-09-20      09/16/2024                 278   
53       3000183284        NaT      09/16/2024                   0   
54       3000183284 2024-09-20      09/16/2024                1932   
55       3000183284        NaT      09/16/2024                   0   
56       3000183284 2024-09-27      09/16/2024                6264   
215      3000177489        NaT      09/16/2024                   0   
216      3000177489        NaT      09/16/2024                   0   
217      3000177489 2024-09-13      09/16/2024                6822   
227      3000183278        NaT      09/16/2024                   0   
228      3000183278 2024-09-13      09/16/2024                 816   
229      3000